In [1]:
# Load MNIST data
load("mnist.RData")

## PCA as preprocessing

In [2]:
# Cast 2D image into 1D array
dim(x_test) <- c(60000, 28 * 28)
dim(x_train) <- c(10000, 28 * 28)

Now we do PCA to the images in order to reduce the dimension of our input data. This preprocessing step would greatly save the time of SVM training. 

In [33]:
pca_results <- princomp(x_train, cor = F)
summary(pca_results)

Importance of components:
                            Comp.1       Comp.2       Comp.3       Comp.4
Standard deviation     587.6084300 509.17881566 459.36501954 431.80500000
Proportion of Variance   0.1004766   0.07544487   0.06140516   0.05425807
Cumulative Proportion    0.1004766   0.17592150   0.23732666   0.29158474
                             Comp.5       Comp.6       Comp.7       Comp.8
Standard deviation     415.80851801 382.00061099 337.33511356 318.41074319
Proportion of Variance   0.05031249   0.04246363   0.03311404   0.02950288
Cumulative Proportion    0.34189722   0.38436086   0.41747490   0.44697778
                             Comp.9      Comp.10      Comp.11      Comp.12
Standard deviation     306.28483042 279.79264052 270.79608231 268.32961726
Proportion of Variance   0.02729858   0.02278041   0.02133899   0.02095204
Cumulative Proportion    0.47427636   0.49705677   0.51839576   0.53934779
                            Comp.13      Comp.14      Comp.15      Comp.16
Sta

We pick the final dimension to be $9$. This is a trade-off between training time and (possibly) prediction accuracy. 

With this dimension, each training costs around $5\,\mathrm{s}$ on my machine. Note that this number will be multiplied by $5$ for 5-fold CV, and another factor of around $10$ for hyperparameter search. 

In [4]:
pca_projections <- as.data.frame(x_train %*% loadings(pca_results)[, 1:9])
pca_projections_test <- as.data.frame(x_test %*% loadings(pca_results)[, 1:9])

In [5]:
dat <- pca_projections
dat['y'] <- as.factor(y_train)

With traning data saved in ``dat``, we are ready to do SVM training. The training will be in the order of linear, radial, and polynomial kernels.  

## SVM training

In this section I will first focus on the training of SVM itself and then show the result of different kernels side by side.  

In [6]:
library(e1071)

### Linear kernel

We directly use $5$-fold CV to choose the best model over ``cost``. The best shot occurs at ``cost=1`` in our case. 

In [32]:
tune.out <- tune(
    svm, y ~ .,
    data = dat, kernel = "linear",
    ranges = list(cost = c(0.01, 0.1, 1, 5, 10)),
    tunecontrol = tune.control(sampling = "cross", cross = 5)
)
summary(tune.out)


Parameter tuning of ‘svm’:

- sampling method: 5-fold cross validation 

- best parameters:
 cost
    1

- best performance: 0.1615 

- Detailed performance results:
   cost  error  dispersion
1  0.01 0.1727 0.005663479
2  0.10 0.1642 0.005251190
3  1.00 0.1615 0.004568917
4  5.00 0.1619 0.004052777
5 10.00 0.1625 0.005431390


In [34]:
svmfit_linear <- svm(y ~ ., data = dat, kernel = "linear", cost = 1)

## Radial kernel

In [42]:
tune.out <- tune(
    svm, y ~ .,
    data = dat, kernel = "radial",
    ranges = list(cost = c(0.1, 1, 5), gamma = c(0.2, 1, 5)), 
    tunecontrol = tune.control(sampling = "cross", cross = 5)
)
summary(tune.out)


Parameter tuning of ‘svm’:

- sampling method: 5-fold cross validation 

- best parameters:
 cost gamma
    5   0.2

- best performance: 0.0829 

- Detailed performance results:
  cost gamma  error  dispersion
1  0.1   0.2 0.1174 0.003507136
2  1.0   0.2 0.0858 0.003581201
3  5.0   0.2 0.0829 0.004292435
4  0.1   1.0 0.3209 0.036705585
5  1.0   1.0 0.0942 0.004957318
6  5.0   1.0 0.0978 0.007285259
7  0.1   5.0 0.8181 0.011414903
8  1.0   5.0 0.5021 0.054820617
9  5.0   5.0 0.4651 0.051705174


In [43]:
svmfit_radial <- svm(y ~ ., data = dat, kernel = "radial", cost = 5, gamma = .2)

### Polynomial kernel

In [44]:
svm(y ~ ., data=dat, kernel='polynomial', cost = 5, degree=3, gamma=.2)


Call:
svm(formula = y ~ ., data = dat, kernel = "polynomial", cost = 1, 
    degree = 3, gamma = 0.3)


Parameters:
   SVM-Type:  C-classification 
 SVM-Kernel:  polynomial 
       cost:  1 
     degree:  3 
     coef.0:  0 

Number of Support Vectors:  2632


In [45]:
tune.out <- tune(
    svm, y ~ .,
    data = dat, kernel = "polynomial",
    cost = 5, 
    gamma = .2,
    ranges = list(degree=1:7), 
    tunecontrol = tune.control(sampling = "cross", cross = 5)
)
summary(tune.out)


Parameter tuning of ‘svm’:

- sampling method: 5-fold cross validation 

- best parameters:
 degree
      3

- best performance: 0.102 

- Detailed performance results:
  degree  error  dispersion
1      1 0.1626 0.007074602
2      2 0.1696 0.008981926
3      3 0.1020 0.008455767
4      4 0.1688 0.007496666
5      5 0.1199 0.006377696
6      6 0.2020 0.004650269
7      7 0.1434 0.008317151


In [47]:
tune.out <- tune(
    svm, y ~ .,
    data = dat, kernel = "polynomial",
    gamma = .2,
    degree = 3,
    ranges = list(cost=c(.1, 1, 5, 10)), 
    tunecontrol = tune.control(sampling = "cross", cross = 5)
)
summary(tune.out)


Parameter tuning of ‘svm’:

- sampling method: 5-fold cross validation 

- best parameters:
 cost
    1

- best performance: 0.0983 

- Detailed performance results:
  cost  error  dispersion
1  0.1 0.1095 0.005326819
2  1.0 0.0983 0.004309872
3  5.0 0.1041 0.004083503
4 10.0 0.1075 0.004000000


In [48]:
tune.out <- tune(
    svm, y ~ .,
    data = dat, kernel = "polynomial",
    degree = 3,
    cost = 1,
    ranges = list(gamma = c(.2, .5, 1, 2, 4)),
    tunecontrol = tune.control(sampling = "cross", cross = 5)
)
summary(tune.out)


Parameter tuning of ‘svm’:

- sampling method: 5-fold cross validation 

- best parameters:
 gamma
   0.2

- best performance: 0.0999 

- Detailed performance results:
  gamma  error  dispersion
1   0.2 0.0999 0.006730527
2   0.5 0.1076 0.010899541
3   1.0 0.1232 0.011421471
4   2.0 0.1296 0.007700649
5   4.0 0.1322 0.009162423


In [51]:
svmfit_polynomial <- svm(
    y ~ .,
    data = dat, kernel = "polynomial", cost = 5, gamma = .2, degree = 3
)

### Prediction summary

In [52]:
y_pred_lin <- predict(svmfit_linear, pca_projections_test)
y_pred_rad <- predict(svmfit_radial, pca_projections_test)
y_pred_poly <- predict(svmfit_polynomial, pca_projections_test)

The accuracy is as follows. 

In [55]:
c(
    linear = mean(y_pred_lin == y_test),
    radial = mean(y_pred_rad == y_test),
    polynomial = mean(y_pred_poly == y_test)
)

linear     radial polynomial 
 0.8233833  0.9069667  0.8851500

## Neural Network

In [69]:
library(tensorflow)
library(keras)

In [65]:
library(tidyverse)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.4     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.0.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



Since ``Keras`` is capable of handling matrix input. We convert the data type back to matrix by just reloading the data file. 

In [60]:
# Load MNIST data
load("mnist.RData")

Next I use $5$-fold CV to find out the best hyperparameter of ``epochs`` and ``batch_size``. The max epoch is set to $10$ (the default value) in compliance with the early-stopping strategy. 

One interesting point to note in the computation is that the larger the ``batch_size``, the quicker NN trains. 

In order to save time finding out how to properly reset a model in ``Keras``, I just copy-and-paste the code $5$ times to simulate the $5$-fold CV needed. 

### MLP

### Batch size $32$

In [101]:
validation_index <- c(1:2000)
model_mlp <- keras_model_sequential()
model_mlp %>%
    layer_flatten(input_shape = c(28, 28)) %>%
    layer_dense(units = 128, activation = "relu") %>%
    layer_dense(units = 10, activation = "softmax")
model_mlp %>% compile(
    optimizer = "adam",
    loss = "sparse_categorical_crossentropy",
    metrics = c("accuracy")
)
metrics <- (model_mlp %>% fit(x_train[-validation_index,,], y_train[-validation_index],
    batch_size=32,
    validation_data=list(x_train[validation_index,,], y_train[validation_index])
))$metrics
valid_acc1 <- metrics$val_accuracy


validation_index <- c(2000:4000)
model_mlp <- keras_model_sequential()
model_mlp %>%
    layer_flatten(input_shape = c(28, 28)) %>%
    layer_dense(units = 128, activation = "relu") %>%
    layer_dense(units = 10, activation = "softmax")
model_mlp %>% compile(
    optimizer = "adam",
    loss = "sparse_categorical_crossentropy",
    metrics = c("accuracy")
)
metrics <- (model_mlp %>% fit(x_train[-validation_index,,], y_train[-validation_index],
    batch_size=32,
    validation_data=list(x_train[validation_index,,], y_train[validation_index])
))$metrics
valid_acc2 <- metrics$val_accuracy

validation_index <- c(4000:6000)
model_mlp <- keras_model_sequential()
model_mlp %>%
    layer_flatten(input_shape = c(28, 28)) %>%
    layer_dense(units = 128, activation = "relu") %>%
    layer_dense(units = 10, activation = "softmax")
model_mlp %>% compile(
    optimizer = "adam",
    loss = "sparse_categorical_crossentropy",
    metrics = c("accuracy")
)
metrics <- (model_mlp %>% fit(x_train[-validation_index,,], y_train[-validation_index],
    batch_size=32,
    validation_data=list(x_train[validation_index,,], y_train[validation_index])
))$metrics
valid_acc3 <- metrics$val_accuracy

validation_index <- c(6000:8000)
model_mlp <- keras_model_sequential()
model_mlp %>%
    layer_flatten(input_shape = c(28, 28)) %>%
    layer_dense(units = 128, activation = "relu") %>%
    layer_dense(units = 10, activation = "softmax")
model_mlp %>% compile(
    optimizer = "adam",
    loss = "sparse_categorical_crossentropy",
    metrics = c("accuracy")
)
metrics <- (model_mlp %>% fit(x_train[-validation_index,,], y_train[-validation_index],
    batch_size=32,
    validation_data=list(x_train[validation_index,,], y_train[validation_index])
))$metrics
valid_acc4 <- metrics$val_accuracy

validation_index <- c(8000:10000)
model_mlp <- keras_model_sequential()
model_mlp %>%
    layer_flatten(input_shape = c(28, 28)) %>%
    layer_dense(units = 128, activation = "relu") %>%
    layer_dense(units = 10, activation = "softmax")
model_mlp %>% compile(
    optimizer = "adam",
    loss = "sparse_categorical_crossentropy",
    metrics = c("accuracy")
)
metrics <- (model_mlp %>% fit(x_train[-validation_index,,], y_train[-validation_index],
    batch_size=32,
    validation_data=list(x_train[validation_index,,], y_train[validation_index])
))$metrics
valid_acc5 <- metrics$val_accuracy

In [106]:
rowMeans(cbind(valid_acc1, valid_acc2, valid_acc3, valid_acc4, valid_acc5))

[1] 0.8551561 0.8810467 0.8964400 0.9053356 0.9051372 0.9145329 0.9105338
 [8] 0.9116334 0.9152319 0.9137304

### Batch size $16$

In [107]:
validation_index <- c(1:2000)
model_mlp <- keras_model_sequential()
model_mlp %>%
    layer_flatten(input_shape = c(28, 28)) %>%
    layer_dense(units = 128, activation = "relu") %>%
    layer_dense(units = 10, activation = "softmax")
model_mlp %>% compile(
    optimizer = "adam",
    loss = "sparse_categorical_crossentropy",
    metrics = c("accuracy")
)
metrics <- (model_mlp %>% fit(x_train[-validation_index,,], y_train[-validation_index],
    batch_size=16,
    validation_data=list(x_train[validation_index,,], y_train[validation_index])
))$metrics
valid_acc1 <- metrics$val_accuracy


validation_index <- c(2000:4000)
model_mlp <- keras_model_sequential()
model_mlp %>%
    layer_flatten(input_shape = c(28, 28)) %>%
    layer_dense(units = 128, activation = "relu") %>%
    layer_dense(units = 10, activation = "softmax")
model_mlp %>% compile(
    optimizer = "adam",
    loss = "sparse_categorical_crossentropy",
    metrics = c("accuracy")
)
metrics <- (model_mlp %>% fit(x_train[-validation_index,,], y_train[-validation_index],
    batch_size=16,
    validation_data=list(x_train[validation_index,,], y_train[validation_index])
))$metrics
valid_acc2 <- metrics$val_accuracy

validation_index <- c(4000:6000)
model_mlp <- keras_model_sequential()
model_mlp %>%
    layer_flatten(input_shape = c(28, 28)) %>%
    layer_dense(units = 128, activation = "relu") %>%
    layer_dense(units = 10, activation = "softmax")
model_mlp %>% compile(
    optimizer = "adam",
    loss = "sparse_categorical_crossentropy",
    metrics = c("accuracy")
)
metrics <- (model_mlp %>% fit(x_train[-validation_index,,], y_train[-validation_index],
    batch_size=16,
    validation_data=list(x_train[validation_index,,], y_train[validation_index])
))$metrics
valid_acc3 <- metrics$val_accuracy

validation_index <- c(6000:8000)
model_mlp <- keras_model_sequential()
model_mlp %>%
    layer_flatten(input_shape = c(28, 28)) %>%
    layer_dense(units = 128, activation = "relu") %>%
    layer_dense(units = 10, activation = "softmax")
model_mlp %>% compile(
    optimizer = "adam",
    loss = "sparse_categorical_crossentropy",
    metrics = c("accuracy")
)
metrics <- (model_mlp %>% fit(x_train[-validation_index,,], y_train[-validation_index],
    batch_size=16,
    validation_data=list(x_train[validation_index,,], y_train[validation_index])
))$metrics
valid_acc4 <- metrics$val_accuracy

validation_index <- c(8000:10000)
model_mlp <- keras_model_sequential()
model_mlp %>%
    layer_flatten(input_shape = c(28, 28)) %>%
    layer_dense(units = 128, activation = "relu") %>%
    layer_dense(units = 10, activation = "softmax")
model_mlp %>% compile(
    optimizer = "adam",
    loss = "sparse_categorical_crossentropy",
    metrics = c("accuracy")
)
metrics <- (model_mlp %>% fit(x_train[-validation_index,,], y_train[-validation_index],
    batch_size=16,
    validation_data=list(x_train[validation_index,,], y_train[validation_index])
))$metrics
valid_acc5 <- metrics$val_accuracy

In [108]:
rowMeans(cbind(valid_acc1, valid_acc2, valid_acc3, valid_acc4, valid_acc5))

[1] 0.8579552 0.8673524 0.8866455 0.8932428 0.9061362 0.9049374 0.8994396
 [8] 0.8951427 0.9108351 0.8995394

### Batch size $64$

In [109]:
validation_index <- c(1:2000)
model_mlp <- keras_model_sequential()
model_mlp %>%
    layer_flatten(input_shape = c(28, 28)) %>%
    layer_dense(units = 128, activation = "relu") %>%
    layer_dense(units = 10, activation = "softmax")
model_mlp %>% compile(
    optimizer = "adam",
    loss = "sparse_categorical_crossentropy",
    metrics = c("accuracy")
)
metrics <- (model_mlp %>% fit(x_train[-validation_index,,], y_train[-validation_index],
    batch_size=64,
    validation_data=list(x_train[validation_index,,], y_train[validation_index])
))$metrics
valid_acc1 <- metrics$val_accuracy


validation_index <- c(2000:4000)
model_mlp <- keras_model_sequential()
model_mlp %>%
    layer_flatten(input_shape = c(28, 28)) %>%
    layer_dense(units = 128, activation = "relu") %>%
    layer_dense(units = 10, activation = "softmax")
model_mlp %>% compile(
    optimizer = "adam",
    loss = "sparse_categorical_crossentropy",
    metrics = c("accuracy")
)
metrics <- (model_mlp %>% fit(x_train[-validation_index,,], y_train[-validation_index],
    batch_size=64,
    validation_data=list(x_train[validation_index,,], y_train[validation_index])
))$metrics
valid_acc2 <- metrics$val_accuracy

validation_index <- c(4000:6000)
model_mlp <- keras_model_sequential()
model_mlp %>%
    layer_flatten(input_shape = c(28, 28)) %>%
    layer_dense(units = 128, activation = "relu") %>%
    layer_dense(units = 10, activation = "softmax")
model_mlp %>% compile(
    optimizer = "adam",
    loss = "sparse_categorical_crossentropy",
    metrics = c("accuracy")
)
metrics <- (model_mlp %>% fit(x_train[-validation_index,,], y_train[-validation_index],
    batch_size=64,
    validation_data=list(x_train[validation_index,,], y_train[validation_index])
))$metrics
valid_acc3 <- metrics$val_accuracy

validation_index <- c(6000:8000)
model_mlp <- keras_model_sequential()
model_mlp %>%
    layer_flatten(input_shape = c(28, 28)) %>%
    layer_dense(units = 128, activation = "relu") %>%
    layer_dense(units = 10, activation = "softmax")
model_mlp %>% compile(
    optimizer = "adam",
    loss = "sparse_categorical_crossentropy",
    metrics = c("accuracy")
)
metrics <- (model_mlp %>% fit(x_train[-validation_index,,], y_train[-validation_index],
    batch_size=64,
    validation_data=list(x_train[validation_index,,], y_train[validation_index])
))$metrics
valid_acc4 <- metrics$val_accuracy

validation_index <- c(8000:10000)
model_mlp <- keras_model_sequential()
model_mlp %>%
    layer_flatten(input_shape = c(28, 28)) %>%
    layer_dense(units = 128, activation = "relu") %>%
    layer_dense(units = 10, activation = "softmax")
model_mlp %>% compile(
    optimizer = "adam",
    loss = "sparse_categorical_crossentropy",
    metrics = c("accuracy")
)
metrics <- (model_mlp %>% fit(x_train[-validation_index,,], y_train[-validation_index],
    batch_size=64,
    validation_data=list(x_train[validation_index,,], y_train[validation_index])
))$metrics
valid_acc5 <- metrics$val_accuracy

In [110]:
rowMeans(cbind(valid_acc1, valid_acc2, valid_acc3, valid_acc4, valid_acc5))

[1] 0.8606508 0.8838440 0.8950396 0.9046357 0.9127324 0.9080342 0.9141329
 [8] 0.9182304 0.9245287 0.9223295

Clearly $5$-fold CV reveals that ``epochs=9`` and ``batch_size=64`` would be a good hyperparameter. The accuracy on test data is $91.1\%$, better than the radial-kernel SVM. 

In [111]:
model_mlp <- keras_model_sequential()
model_mlp %>%
    layer_flatten(input_shape = c(28, 28)) %>%
    layer_dense(units = 128, activation = "relu") %>%
    layer_dense(units = 10, activation = "softmax")
model_mlp %>% compile(
    optimizer = "adam",
    loss = "sparse_categorical_crossentropy",
    metrics = c("accuracy")
)
model_mlp %>% fit(x_train, y_train,
    batch_size = 64,
    epochs = 9,
    validation_split = .2
)
score <- model_mlp %>% evaluate(x_test, y_test)

cat("Test loss:", score[1], "\n")
cat("Test accuracy:", score[2], "\n")

Test loss: 2.921334 
Test accuracy: 0.9113 


### CNN

In [113]:
x_train.cnn <- array(x_train, dim = c(dim(x_train)[1], dim(x_train)[2], dim(x_train)[3], 1))
x_test.cnn <- array(x_test, dim = c(dim(x_test)[1], dim(x_test)[2], dim(x_test)[3], 1))

### Batch size $32$

In [123]:
validation_index <- c(1:2000)
model.cnn <- keras_model_sequential()
# configuring the Model
model.cnn %>%
    layer_conv_2d(filter = 32, kernel_size = c(3, 3), padding = "same", input_shape = c(28, 28, 1)) %>%
    layer_activation("relu") %>%
    layer_max_pooling_2d(pool_size = c(2, 2)) %>%
    layer_conv_2d(filter = 32, kernel_size = c(3, 3)) %>%
    layer_activation("relu") %>%
    layer_max_pooling_2d(pool_size = c(2, 2)) %>%
    layer_dropout(0.25) %>%
    layer_flatten() %>%
    layer_dense(64) %>%
    layer_activation("relu") %>%
    layer_dropout(0.5) %>%
    layer_dense(10) %>%
    layer_activation("softmax")
model.cnn %>% compile(
  optimizer = 'adam', 
  loss = 'sparse_categorical_crossentropy',
  metrics = c('accuracy')
)
metrics <- (model.cnn %>% fit(x_train[-validation_index,,], y_train[-validation_index],
    batch_size=32,
    validation_data=list(x_train[validation_index,,], y_train[validation_index])
))$metrics
valid_acc1 <- metrics$val_accuracy

validation_index <- c(2000:4000)
model.cnn <- keras_model_sequential()
# configuring the Model
model.cnn %>%
    layer_conv_2d(filter = 32, kernel_size = c(3, 3), padding = "same", input_shape = c(28, 28, 1)) %>%
    layer_activation("relu") %>%
    layer_max_pooling_2d(pool_size = c(2, 2)) %>%
    layer_conv_2d(filter = 32, kernel_size = c(3, 3)) %>%
    layer_activation("relu") %>%
    layer_max_pooling_2d(pool_size = c(2, 2)) %>%
    layer_dropout(0.25) %>%
    layer_flatten() %>%
    layer_dense(64) %>%
    layer_activation("relu") %>%
    layer_dropout(0.5) %>%
    layer_dense(10) %>%
    layer_activation("softmax")
model.cnn %>% compile(
  optimizer = 'adam', 
  loss = 'sparse_categorical_crossentropy',
  metrics = c('accuracy')
)
metrics <- (model.cnn %>% fit(x_train[-validation_index,,], y_train[-validation_index],
    batch_size=32,
    validation_data=list(x_train[validation_index,,], y_train[validation_index])
))$metrics
valid_acc2 <- metrics$val_accuracy

validation_index <- c(4000:6000)
model.cnn <- keras_model_sequential()
# configuring the Model
model.cnn %>%
    layer_conv_2d(filter = 32, kernel_size = c(3, 3), padding = "same", input_shape = c(28, 28, 1)) %>%
    layer_activation("relu") %>%
    layer_max_pooling_2d(pool_size = c(2, 2)) %>%
    layer_conv_2d(filter = 32, kernel_size = c(3, 3)) %>%
    layer_activation("relu") %>%
    layer_max_pooling_2d(pool_size = c(2, 2)) %>%
    layer_dropout(0.25) %>%
    layer_flatten() %>%
    layer_dense(64) %>%
    layer_activation("relu") %>%
    layer_dropout(0.5) %>%
    layer_dense(10) %>%
    layer_activation("softmax")
model.cnn %>% compile(
  optimizer = 'adam', 
  loss = 'sparse_categorical_crossentropy',
  metrics = c('accuracy')
)
metrics <- (model.cnn %>% fit(x_train[-validation_index,,], y_train[-validation_index],
    batch_size=32,
    validation_data=list(x_train[validation_index,,], y_train[validation_index])
))$metrics
valid_acc3 <- metrics$val_accuracy

validation_index <- c(6000:8000)
model.cnn <- keras_model_sequential()
# configuring the Model
model.cnn %>%
    layer_conv_2d(filter = 32, kernel_size = c(3, 3), padding = "same", input_shape = c(28, 28, 1)) %>%
    layer_activation("relu") %>%
    layer_max_pooling_2d(pool_size = c(2, 2)) %>%
    layer_conv_2d(filter = 32, kernel_size = c(3, 3)) %>%
    layer_activation("relu") %>%
    layer_max_pooling_2d(pool_size = c(2, 2)) %>%
    layer_dropout(0.25) %>%
    layer_flatten() %>%
    layer_dense(64) %>%
    layer_activation("relu") %>%
    layer_dropout(0.5) %>%
    layer_dense(10) %>%
    layer_activation("softmax")
model.cnn %>% compile(
  optimizer = 'adam', 
  loss = 'sparse_categorical_crossentropy',
  metrics = c('accuracy')
)
metrics <- (model.cnn %>% fit(x_train[-validation_index,,], y_train[-validation_index],
    batch_size=32,
    validation_data=list(x_train[validation_index,,], y_train[validation_index])
))$metrics
valid_acc4 <- metrics$val_accuracy


validation_index <- c(8000:10000)
model.cnn <- keras_model_sequential()
# configuring the Model
model.cnn %>%
    layer_conv_2d(filter = 32, kernel_size = c(3, 3), padding = "same", input_shape = c(28, 28, 1)) %>%
    layer_activation("relu") %>%
    layer_max_pooling_2d(pool_size = c(2, 2)) %>%
    layer_conv_2d(filter = 32, kernel_size = c(3, 3)) %>%
    layer_activation("relu") %>%
    layer_max_pooling_2d(pool_size = c(2, 2)) %>%
    layer_dropout(0.25) %>%
    layer_flatten() %>%
    layer_dense(64) %>%
    layer_activation("relu") %>%
    layer_dropout(0.5) %>%
    layer_dense(10) %>%
    layer_activation("softmax")
model.cnn %>% compile(
  optimizer = 'adam', 
  loss = 'sparse_categorical_crossentropy',
  metrics = c('accuracy')
)
metrics <- (model.cnn %>% fit(x_train[-validation_index,,], y_train[-validation_index],
    batch_size=32,
    validation_data=list(x_train[validation_index,,], y_train[validation_index])
))$metrics
valid_acc5 <- metrics$val_accuracy

In [124]:
rowMeans(cbind(valid_acc1, valid_acc2, valid_acc3, valid_acc4, valid_acc5))

[1] 0.6806941 0.8175338 0.8936215 0.9306189 0.9495141 0.9567142 0.9617130
 [8] 0.9646124 0.9709102 0.9697104

### Batch size $64$

In [119]:
validation_index <- c(1:2000)
model.cnn <- keras_model_sequential()
# configuring the Model
model.cnn %>%
    layer_conv_2d(filter = 32, kernel_size = c(3, 3), padding = "same", input_shape = c(28, 28, 1)) %>%
    layer_activation("relu") %>%
    layer_max_pooling_2d(pool_size = c(2, 2)) %>%
    layer_conv_2d(filter = 32, kernel_size = c(3, 3)) %>%
    layer_activation("relu") %>%
    layer_max_pooling_2d(pool_size = c(2, 2)) %>%
    layer_dropout(0.25) %>%
    layer_flatten() %>%
    layer_dense(64) %>%
    layer_activation("relu") %>%
    layer_dropout(0.5) %>%
    layer_dense(10) %>%
    layer_activation("softmax")
model.cnn %>% compile(
  optimizer = 'adam', 
  loss = 'sparse_categorical_crossentropy',
  metrics = c('accuracy')
)
metrics <- (model.cnn %>% fit(x_train[-validation_index,,], y_train[-validation_index],
    batch_size=64,
    validation_data=list(x_train[validation_index,,], y_train[validation_index])
))$metrics
valid_acc1 <- metrics$val_accuracy

validation_index <- c(2000:4000)
model.cnn <- keras_model_sequential()
# configuring the Model
model.cnn %>%
    layer_conv_2d(filter = 32, kernel_size = c(3, 3), padding = "same", input_shape = c(28, 28, 1)) %>%
    layer_activation("relu") %>%
    layer_max_pooling_2d(pool_size = c(2, 2)) %>%
    layer_conv_2d(filter = 32, kernel_size = c(3, 3)) %>%
    layer_activation("relu") %>%
    layer_max_pooling_2d(pool_size = c(2, 2)) %>%
    layer_dropout(0.25) %>%
    layer_flatten() %>%
    layer_dense(64) %>%
    layer_activation("relu") %>%
    layer_dropout(0.5) %>%
    layer_dense(10) %>%
    layer_activation("softmax")
model.cnn %>% compile(
  optimizer = 'adam', 
  loss = 'sparse_categorical_crossentropy',
  metrics = c('accuracy')
)
metrics <- (model.cnn %>% fit(x_train[-validation_index,,], y_train[-validation_index],
    batch_size=64,
    validation_data=list(x_train[validation_index,,], y_train[validation_index])
))$metrics
valid_acc2 <- metrics$val_accuracy

validation_index <- c(4000:6000)
model.cnn <- keras_model_sequential()
# configuring the Model
model.cnn %>%
    layer_conv_2d(filter = 32, kernel_size = c(3, 3), padding = "same", input_shape = c(28, 28, 1)) %>%
    layer_activation("relu") %>%
    layer_max_pooling_2d(pool_size = c(2, 2)) %>%
    layer_conv_2d(filter = 32, kernel_size = c(3, 3)) %>%
    layer_activation("relu") %>%
    layer_max_pooling_2d(pool_size = c(2, 2)) %>%
    layer_dropout(0.25) %>%
    layer_flatten() %>%
    layer_dense(64) %>%
    layer_activation("relu") %>%
    layer_dropout(0.5) %>%
    layer_dense(10) %>%
    layer_activation("softmax")
model.cnn %>% compile(
  optimizer = 'adam', 
  loss = 'sparse_categorical_crossentropy',
  metrics = c('accuracy')
)
metrics <- (model.cnn %>% fit(x_train[-validation_index,,], y_train[-validation_index],
    batch_size=64,
    validation_data=list(x_train[validation_index,,], y_train[validation_index])
))$metrics
valid_acc3 <- metrics$val_accuracy

validation_index <- c(6000:8000)
model.cnn <- keras_model_sequential()
# configuring the Model
model.cnn %>%
    layer_conv_2d(filter = 32, kernel_size = c(3, 3), padding = "same", input_shape = c(28, 28, 1)) %>%
    layer_activation("relu") %>%
    layer_max_pooling_2d(pool_size = c(2, 2)) %>%
    layer_conv_2d(filter = 32, kernel_size = c(3, 3)) %>%
    layer_activation("relu") %>%
    layer_max_pooling_2d(pool_size = c(2, 2)) %>%
    layer_dropout(0.25) %>%
    layer_flatten() %>%
    layer_dense(64) %>%
    layer_activation("relu") %>%
    layer_dropout(0.5) %>%
    layer_dense(10) %>%
    layer_activation("softmax")
model.cnn %>% compile(
  optimizer = 'adam', 
  loss = 'sparse_categorical_crossentropy',
  metrics = c('accuracy')
)
metrics <- (model.cnn %>% fit(x_train[-validation_index,,], y_train[-validation_index],
    batch_size=64,
    validation_data=list(x_train[validation_index,,], y_train[validation_index])
))$metrics
valid_acc4 <- metrics$val_accuracy


validation_index <- c(8000:10000)
model.cnn <- keras_model_sequential()
# configuring the Model
model.cnn %>%
    layer_conv_2d(filter = 32, kernel_size = c(3, 3), padding = "same", input_shape = c(28, 28, 1)) %>%
    layer_activation("relu") %>%
    layer_max_pooling_2d(pool_size = c(2, 2)) %>%
    layer_conv_2d(filter = 32, kernel_size = c(3, 3)) %>%
    layer_activation("relu") %>%
    layer_max_pooling_2d(pool_size = c(2, 2)) %>%
    layer_dropout(0.25) %>%
    layer_flatten() %>%
    layer_dense(64) %>%
    layer_activation("relu") %>%
    layer_dropout(0.5) %>%
    layer_dense(10) %>%
    layer_activation("softmax")
model.cnn %>% compile(
  optimizer = 'adam', 
  loss = 'sparse_categorical_crossentropy',
  metrics = c('accuracy')
)
metrics <- (model.cnn %>% fit(x_train[-validation_index,,], y_train[-validation_index],
    batch_size=64,
    validation_data=list(x_train[validation_index,,], y_train[validation_index])
))$metrics
valid_acc5 <- metrics$val_accuracy

In [120]:
rowMeans(cbind(valid_acc1, valid_acc2, valid_acc3, valid_acc4, valid_acc5))

[1] 0.5543768 0.7844881 0.8954386 0.9372229 0.9500183 0.9581154 0.9601146
 [8] 0.9625144 0.9669125 0.9668120

### Batch size $128$ 

In [121]:
validation_index <- c(1:2000)
model.cnn <- keras_model_sequential()
# configuring the Model
model.cnn %>%
    layer_conv_2d(filter = 32, kernel_size = c(3, 3), padding = "same", input_shape = c(28, 28, 1)) %>%
    layer_activation("relu") %>%
    layer_max_pooling_2d(pool_size = c(2, 2)) %>%
    layer_conv_2d(filter = 32, kernel_size = c(3, 3)) %>%
    layer_activation("relu") %>%
    layer_max_pooling_2d(pool_size = c(2, 2)) %>%
    layer_dropout(0.25) %>%
    layer_flatten() %>%
    layer_dense(64) %>%
    layer_activation("relu") %>%
    layer_dropout(0.5) %>%
    layer_dense(10) %>%
    layer_activation("softmax")
model.cnn %>% compile(
  optimizer = 'adam', 
  loss = 'sparse_categorical_crossentropy',
  metrics = c('accuracy')
)
metrics <- (model.cnn %>% fit(x_train[-validation_index,,], y_train[-validation_index],
    batch_size=128,
    validation_data=list(x_train[validation_index,,], y_train[validation_index])
))$metrics
valid_acc1 <- metrics$val_accuracy

validation_index <- c(2000:4000)
model.cnn <- keras_model_sequential()
# configuring the Model
model.cnn %>%
    layer_conv_2d(filter = 32, kernel_size = c(3, 3), padding = "same", input_shape = c(28, 28, 1)) %>%
    layer_activation("relu") %>%
    layer_max_pooling_2d(pool_size = c(2, 2)) %>%
    layer_conv_2d(filter = 32, kernel_size = c(3, 3)) %>%
    layer_activation("relu") %>%
    layer_max_pooling_2d(pool_size = c(2, 2)) %>%
    layer_dropout(0.25) %>%
    layer_flatten() %>%
    layer_dense(64) %>%
    layer_activation("relu") %>%
    layer_dropout(0.5) %>%
    layer_dense(10) %>%
    layer_activation("softmax")
model.cnn %>% compile(
  optimizer = 'adam', 
  loss = 'sparse_categorical_crossentropy',
  metrics = c('accuracy')
)
metrics <- (model.cnn %>% fit(x_train[-validation_index,,], y_train[-validation_index],
    batch_size=128,
    validation_data=list(x_train[validation_index,,], y_train[validation_index])
))$metrics
valid_acc2 <- metrics$val_accuracy

validation_index <- c(4000:6000)
model.cnn <- keras_model_sequential()
# configuring the Model
model.cnn %>%
    layer_conv_2d(filter = 32, kernel_size = c(3, 3), padding = "same", input_shape = c(28, 28, 1)) %>%
    layer_activation("relu") %>%
    layer_max_pooling_2d(pool_size = c(2, 2)) %>%
    layer_conv_2d(filter = 32, kernel_size = c(3, 3)) %>%
    layer_activation("relu") %>%
    layer_max_pooling_2d(pool_size = c(2, 2)) %>%
    layer_dropout(0.25) %>%
    layer_flatten() %>%
    layer_dense(64) %>%
    layer_activation("relu") %>%
    layer_dropout(0.5) %>%
    layer_dense(10) %>%
    layer_activation("softmax")
model.cnn %>% compile(
  optimizer = 'adam', 
  loss = 'sparse_categorical_crossentropy',
  metrics = c('accuracy')
)
metrics <- (model.cnn %>% fit(x_train[-validation_index,,], y_train[-validation_index],
    batch_size=128,
    validation_data=list(x_train[validation_index,,], y_train[validation_index])
))$metrics
valid_acc3 <- metrics$val_accuracy

validation_index <- c(6000:8000)
model.cnn <- keras_model_sequential()
# configuring the Model
model.cnn %>%
    layer_conv_2d(filter = 32, kernel_size = c(3, 3), padding = "same", input_shape = c(28, 28, 1)) %>%
    layer_activation("relu") %>%
    layer_max_pooling_2d(pool_size = c(2, 2)) %>%
    layer_conv_2d(filter = 32, kernel_size = c(3, 3)) %>%
    layer_activation("relu") %>%
    layer_max_pooling_2d(pool_size = c(2, 2)) %>%
    layer_dropout(0.25) %>%
    layer_flatten() %>%
    layer_dense(64) %>%
    layer_activation("relu") %>%
    layer_dropout(0.5) %>%
    layer_dense(10) %>%
    layer_activation("softmax")
model.cnn %>% compile(
  optimizer = 'adam', 
  loss = 'sparse_categorical_crossentropy',
  metrics = c('accuracy')
)
metrics <- (model.cnn %>% fit(x_train[-validation_index,,], y_train[-validation_index],
    batch_size=128,
    validation_data=list(x_train[validation_index,,], y_train[validation_index])
))$metrics
valid_acc4 <- metrics$val_accuracy


validation_index <- c(8000:10000)
model.cnn <- keras_model_sequential()
# configuring the Model
model.cnn %>%
    layer_conv_2d(filter = 32, kernel_size = c(3, 3), padding = "same", input_shape = c(28, 28, 1)) %>%
    layer_activation("relu") %>%
    layer_max_pooling_2d(pool_size = c(2, 2)) %>%
    layer_conv_2d(filter = 32, kernel_size = c(3, 3)) %>%
    layer_activation("relu") %>%
    layer_max_pooling_2d(pool_size = c(2, 2)) %>%
    layer_dropout(0.25) %>%
    layer_flatten() %>%
    layer_dense(64) %>%
    layer_activation("relu") %>%
    layer_dropout(0.5) %>%
    layer_dense(10) %>%
    layer_activation("softmax")
model.cnn %>% compile(
  optimizer = 'adam', 
  loss = 'sparse_categorical_crossentropy',
  metrics = c('accuracy')
)
metrics <- (model.cnn %>% fit(x_train[-validation_index,,], y_train[-validation_index],
    batch_size=128,
    validation_data=list(x_train[validation_index,,], y_train[validation_index])
))$metrics
valid_acc5 <- metrics$val_accuracy

In [122]:
rowMeans(cbind(valid_acc1, valid_acc2, valid_acc3, valid_acc4, valid_acc5))

[1] 0.3916586 0.6550521 0.7883937 0.8633590 0.9189331 0.9338256 0.9469204
 [8] 0.9529181 0.9588159 0.9611148

This time, the best accuracy occurs at ``batch_size=32`` and ``epochs=9``. Although training CNN is more time-consuming, an accuracy of $96.2\%$ makes it worthwhile. 

In [126]:
model.cnn <- keras_model_sequential()
# configuring the Model
model.cnn %>%
    layer_conv_2d(filter = 32, kernel_size = c(3, 3), padding = "same", input_shape = c(28, 28, 1)) %>%
    layer_activation("relu") %>%
    layer_max_pooling_2d(pool_size = c(2, 2)) %>%
    layer_conv_2d(filter = 32, kernel_size = c(3, 3)) %>%
    layer_activation("relu") %>%
    layer_max_pooling_2d(pool_size = c(2, 2)) %>%
    layer_dropout(0.25) %>%
    layer_flatten() %>%
    layer_dense(64) %>%
    layer_activation("relu") %>%
    layer_dropout(0.5) %>%
    layer_dense(10) %>%
    layer_activation("softmax")
model.cnn %>% compile(
    optimizer = "adam",
    loss = "sparse_categorical_crossentropy",
    metrics = c("accuracy")
)
model.cnn %>% fit(x_train[-validation_index, , ], y_train[-validation_index],
    batch_size = 32,
    validation_split = 0.2,
    epochs = 9
)
score <- model.cnn %>% evaluate(x_test, y_test)

cat("Test loss:", score[1], "\n")
cat("Test accuracy:", score[2], "\n")

Test loss: 0.1349014 
Test accuracy: 0.9622333 


To sum up, I would like to resummarise the results into the following table. 

In [133]:
data.frame(
    method = c("Linear SVM", "Radial SVM", "Polynomial SVM", "MLP", "CNN"),
    best.accuracy = c(0.823, .907, .885, .911, .962),
    optimized.parameter = c("cost=1", "cost=5, gamma=0.2", "cost=1, gamma=0.2, degree=3", "batch size=64, epochs=9", "batch size=32, epochs=9")
)

method,best.accuracy,optimized.parameter
<chr>,<dbl>,<chr>
Linear SVM,0.823,cost=1
Radial SVM,0.907,"cost=5, gamma=0.2"
Polynomial SVM,0.885,"cost=1, gamma=0.2, degree=3"
MLP,0.911,"batch size=64, epochs=9"
CNN,0.962,"batch size=32, epochs=9"
